In [95]:
# Imports
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)

In [2]:
master_data = pd.read_csv("../HumanObserved-Dataset/HumanObserved-Features-Data/HumanObserved-Features-Data.csv")
pos_data = pd.read_csv("../HumanObserved-Dataset/HumanObserved-Features-Data/same_pairs.csv")

In [4]:
def create_setting_one(master_data,pos_data):
    raw_data_temp = pd.concat([pos_data.set_index('img_id_A'),master_data.set_index('img_id')],axis=1,join='inner').reset_index()
    raw_data_feature_concat = pd.concat([raw_data_temp.set_index('img_id_B'),master_data.set_index('img_id')],axis=1,join='inner').reset_index()
    raw_data_feature_concat.drop(['Unnamed: 0'],axis=1,inplace=True)
    col_rename = ['img_id_B','img_id_A','target']
    for columns in range(1,len(list(raw_data_feature_concat.columns)[3:])+1):
        if(columns < 10):
            col_rename.append("fa"+str(columns))
        else:
            col_rename.append("fb"+str(columns - 9))
    raw_data_feature_concat.columns = col_rename
    col_rename.append(col_rename.pop(2))
    temp = col_rename[0]
    col_rename[0] = col_rename[1]
    col_rename[1] = temp
    raw_data_feature_concat = raw_data_feature_concat[col_rename]
    return raw_data_feature_concat

def create_setting_two(raw_data_feature_concat):
    raw_data_feature_subs = raw_data_feature_concat.copy()
    for columns in range(1,int((len(list(raw_data_feature_subs.columns))-3)/2+1)):
        raw_data_feature_subs['fm'+str(columns)] = abs(raw_data_feature_subs['fa'+str(columns)] - raw_data_feature_subs['fb'+str(columns)])
        raw_data_feature_subs.drop('fa'+str(columns),axis=1,inplace=True)
        raw_data_feature_subs.drop('fb'+str(columns),axis=1,inplace=True)
    col_swap = list(raw_data_feature_subs.columns)
    col_swap.append(col_swap.pop(2))
    raw_data_feature_subs=raw_data_feature_subs[col_swap]
    return raw_data_feature_subs

In [102]:
raw_data_feature_concat = create_setting_one(master_data,pos_data)
raw_data_feature_subs = create_setting_two(raw_data_feature_concat)

In [ ]:
# Unseen Writer partitions

raw_data_feature_concat[['A','A_imgNo']] = raw_data_feature_concat['img_id_A'].str.extract('(\d\d\d\d)([a-z])', expand=False)
raw_data_feature_concat[['B','B_imgNo']] = raw_data_feature_concat['img_id_B'].str.extract('(\d\d\d\d)([a-z])', expand=False)
#raw_data_feature_concat['img_id_A'].str.extract('(?P<writerA>\d\d\d\d)(?P<imageNo>[abcd])', expand=False)

array([False])